![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

**Please restart the kernel. It's mandatory to get everything up and running.**

To save time, we prepared one catalog for you. Feel free to see the format

In [ ]:
import os
from obspy import read_events
from SeisMonitor.utils4examples import clone_seismonitor_data

monitor_path = os.path.join(os.path.dirname(os.getcwd()),"6m")
out_dir = os.path.join(monitor_path,"Ml")
others = monitor_path = os.path.join(os.path.dirname(os.getcwd()),"others")

clone_seismonitor_data(others,branch="others")
events = os.path.join(others,"events")
eqt_events = os.path.join(events,"eqt")
catalog = os.path.join(eqt_events,"associations.xml")


print("events dir: ",events)
print("Important folders in events directory",os.listdir(events))
print("#"*100)
print("eqt_events dir: ",eqt_events)
print("\tImportant files in eqt_events directory",os.listdir(eqt_events))
print("Catalog: ",read_events(catalog,format="QUAKEML"))

# Magnitude

In [1]:
import os
import math
from obspy.clients.fdsn import Client as FDSNClient
from obspy.core.utcdatetime import UTCDateTime
from obspy.core.inventory.inventory import read_inventory
from SeisMonitor.monitor.magnitude.mag import Magnitude,MwPhysicalMagParams, MwProcessingMagParams
from SeisMonitor.core.objects import WaveformRestrictions,Provider

We are going to compute local magnitude ${M}_{l}$ in our catalog. We first need to define three main parameters

- **catalog**: XML Path or Catalog object which contains the events 
- **Provider**: Please take a look in [Downloader](https://colab.research.google.com/github/ecastillot/SeisMonitor/blob/master/examples/1.downloader.ipynb) to know how to instance it.
- **out_dir**: Output directory to see your results

In [ ]:
sgc_client = FDSNClient('http://sismo.sgc.gov.co:8080')
sgc_rest = WaveformRestrictions(network="CM",
                    station="*",
                    location="*",
                    channel="*",
                    starttime=UTCDateTime("2019-12-24T19:00:00.000000Z"), # starttime of your events. 
                    endtime=UTCDateTime("2019-12-25T01:00:00.000000Z"), # endtime of your events.
                    location_preferences=["","00","20","10","40"],
                    channel_preferences=["HH","BH","EH","HN","HL"],
                    filter_networks=[], 
                    filter_stations=[],
                    filter_domain= [-83.101,-64.549,-2.229,14.945],
                    )
sgc_client = FDSNClient('http://sismo.sgc.gov.co:8080')
sgc_provider = Provider(sgc_client,sgc_rest)


In [ ]:
mag = Magnitude([sgc_provider],catalog,out_dir) #catalog,providers,out

## ${M}_{l}$

$$ {M}_{l}=logA+a*log(r/{r}_{ref} )+b*log(r-{r}_{ref} )+K({r}_{ref})$$

- maximum amplitude A in millimeters on the two horizontal components simulating a Wood-Anderson seismograph
- a=1.019 related to the geometrical spreading in Colombia
- b=0.0016 related to the anelastic attenuation factor in Colombia
- ${r}_{ref}=140 km$ is the reference distance  in Colombia that stablishes a common point with the traditional Richter scale in California.
- $K({r}_{ref})$ is the base level or the reference magnitude when ${M}_{l}=3 km$

Values for Colombia are taken from Rengifo and Carriazo (2004)


Preparing this equation

In [3]:
ml_params = {"a":1.019,"b":0.0016,"r_ref":140} #ojeda

k = ml_params["a"]*math.log10(ml_params["r_ref"]/100) +\
                    ml_params["b"]* (ml_params["r_ref"]-100) +3

Ml = lambda ampl,epi_dist : math.log10(ampl * 1e3) + ml_params["a"] * math.log10(epi_dist/ml_params["r_ref"]) +\
                                ml_params["b"] * (epi_dist-ml_params["r_ref"]) + k

Computing the magnitude

In [ ]:
cat = mag.get_Ml(mag_type=Ml ,
            trimmedtime=5, #seconds after pick S to trim the signal
            out_format="SC3ML")
print(cat)